In [1]:


# año, torneo, nombre, edad,altura, ranking_ATP_inicial, minutos_jugados_acumulado_hist,
# pj_acum_historicos, pg_acum_historico, pp_acum_historico, titulos_ganados_acum, ultima_modificacion, posicion_final, score


# 2014,  roland_garros, rafa, 7, 1, 7, 0, sum(minutos), fijo, 6, clay, 1, 100, 2





# predecir 2015

# 1 -> 32/32=1
# 2 -> 31/32=0.99

In [2]:
# Use this initial code to work in the notebook as if it were a module, that 
# is, to be able to export classes and functions from other subpackages.

import pyprojroot
import sys

package_path = pyprojroot.here().__str__()
if package_path not in sys.path:
    sys.path.append(package_path)

In [3]:
clean_data = 'MatchesTable.csv'
clean_dataplayers = 'PlayerTable.csv'

In [18]:
from utils import LoadData, AbsPaths
import pandas as pd

In [5]:
df = LoadData().from_csv(file_name=clean_data)
players=LoadData().from_csv(file_name=clean_dataplayers)
df.head(2)

,tourney_id,tourney_name,surface,tourney_date,round,winner_id,winner_rank,winner_rank_points,loser_id,loser_rank,loser_rank_points,score,minutes
0,2000-560,US Open,Hard,20000828.0,R128,101736.0,1.0,3754.0,103297.0,432.0,53.0,6-4 6-2 6-0,89.0
1,2000-560,US Open,Hard,20000828.0,R128,103096.0,37.0,928.0,103292.0,26.0,1067.0,6-3 6-4 6-2,105.0


In [6]:
def organize_data_type(data: pd.DataFrame, data_type: str):
    df = data.copy()

    cols = [
        "tourney_id",
        "tourney_name",
        "surface",
        "tourney_date",
        f"{data_type}_id",
        f"{data_type}_rank",
        "minutes",
        "round",
    ]

    rounds = df["round"].unique()
    change_round = {
        r: int(r.split("R")[-1])
        if r.startswith("R")
        else 4
        if r.startswith("Q")
        else 2
        if r.startswith("S")
        else 1
        if r.startswith("F")
        else 0
        for r in rounds
    }
    df["round"].replace(change_round, inplace=True)

    rename_cols = {col: col.split("_")[-1] for col in cols if col.startswith(data_type)}
    df["tourney_date"] = pd.to_datetime(df["tourney_date"])

    types = {
        "tourney_id": str,
        "tourney_name": str,
        "surface": str,
        "id": str,
        "rank": float,
        "minutes": float,
    }

    return df.loc[:, cols].rename(columns=rename_cols).astype(types)



In [7]:
def organize_original_data(data):
    original_data = data.copy()

    original_data.tourney_date = pd.to_datetime(
        original_data.tourney_date.astype(int), format="%Y%m%d"
    )

    winner_df = organize_data_type(data=original_data, data_type="winner")
    winner_df["won"] = 1
    loser_df = organize_data_type(data=original_data, data_type="loser")
    loser_df["won"] = 0

    final_df = pd.concat([winner_df, loser_df])
    final_df.set_index("tourney_date", inplace=True)
    return final_df


#final_df = organize_original_data(data=df)


In [24]:
def calc_historical_features(row, history):
    df = history.loc[
        (history["tourney_name"] == row["tourney_name"])
        & (history["id"] == row["id"]),
        ["tourney_name", "id", "won", "minutes"],
    ].copy()

    agg_dictionary = {"won": ["count", "sum"], "minutes": "sum"}

    df = df.groupby(by=["tourney_name", "id"]).agg(agg_dictionary).reset_index()
    df.columns = df.columns.map(lambda x: "".join(map(str, x)))
    df.rename(
        columns={"woncount": "pj", "wonsum": "pg", "minutessum": "minutes_played"},
        inplace=True,
    )
    df["pp"] = df["pj"] - df["pg"]

    return df


def make_features(data: pd.DataFrame, players:pd.DataFrame, start_year: int = None, end_year: int = None):
    
    # Organize data
    organized_data = organize_original_data(data=data)

    if start_year is None:
        start_year = organized_data.index.year.min()
    if end_year is None:
        end_year = organized_data.index.year.max()

    final_data = pd.DataFrame()

    # Calculate features for each year
    for year in range(start_year, end_year + 1):
        print(year)

        # Current features
        current_tourneys = organized_data.sort_index().loc[str(year)].reset_index()


        unique_cols = [
            "tourney_name",
            "tourney_date",
            "surface",
            "tourney_id",
            "id"
        ]

        group = current_tourneys.groupby(by=unique_cols).agg(
            {"rank": "max", "round": ["min"]}
        )

        current_features = group.reset_index()
        current_features.columns = current_features.columns.droplevel(1)

        # Historical features
        history = organized_data.sort_index().loc[: str(year - 1)]
        historical_features = pd.concat(
            current_features.apply(
                lambda row: calc_historical_features(row, history), axis=1
            ).to_list(),
            ignore_index=True,
        )

        full_features = current_features.merge(historical_features, how="left", on=["tourney_name", "id"])
        full_features = full_features.fillna(
            value={"pj": 0, "pg": 0, "pp": 0, "minutes_played": 0}
        )
        full_features["year"] = year        
        
        

        final_data = pd.concat([final_data, full_features], ignore_index=True)
        
        for i in set(final_data.id):
            final_data.loc[final_data.id == i, "ht"] = players[players.id == float(i)]["ht"].values[0]
            final_data.loc[final_data.id == i, "wg"] = players[players.id == float(i)]["weight_kg"].values[0]
            final_data.loc[final_data.id == i, "birthdate"] = players[players.id == float(i)]["birthdate"].values[0]
        
        final_data.birthdate = pd.to_datetime(final_data.birthdate)
        final_data.loc[:,"age"] = final_data.loc[:,"year"] - final_data.loc[:,"birthdate"].dt.year

        # Sort output data
        out_cols = [
            "tourney_date",
            "tourney_name",
            "surface",
            "id",
            "ht",
            "wg",
            "age",
            "rank",
            "minutes_played",
            "pj",
            "pg",
            "pp",
            "round",
        ]

    processed_path = AbsPaths().get_abs_path_folder(folder_name="processed")

    return final_data.loc[:, out_cols]
    # final_data.to_csv(processed_path + "operational_rdbms.csv", index=False)


data_features = make_features(data=df, players=players, start_year = 2017, end_year=2021)
data_features

2017
2018


KeyError: '2018'

In [ ]:
def verify_historic_dates(file_name:str = "atp_matches_historic"):
    interim_path = AbsPaths().get_abs_path_folder(folder_name="interim")
    data_hist = pd.read_csv(interim_path + f"{file_name}")

In [17]:
def verify_rdbms(file_name:str = "operational_rdbms", start_year = 2000, end_year=2017):
    
    file_name = file_name + ".csv"
    processed_path = AbsPaths().get_abs_path_folder(folder_name="processed")
    try:
        data = pd.read_csv(processed_path + file_name, parse_dates="tourney_date")
        last_year = data.tourney_date.dt.year.max()
        data2 = make_features(start_year=last_year + 1, end_year = end_year)
        features = pd.concat([data, data2], ignore_index=True)
    except:
        features = make_features(start_year=start_year, end_year=end_year)

    features.to_csv(processed_path + "operational_rdbms.csv", index=False)


    



,year,tourney_name,surface,id,ht,wg,age,rank,minutes_played,pj,pg,pp,round
0,2000,Australian Open,Hard,101086.0,180.000000,82.0,36,94.0,0.0,0.0,0.0,0.0,128
1,2000,Australian Open,Hard,101150.0,180.000000,80.0,35,82.0,0.0,0.0,0.0,0.0,64
2,2000,Australian Open,Hard,101185.0,188.000000,88.0,35,126.0,0.0,0.0,0.0,0.0,64
3,2000,Australian Open,Hard,101463.0,185.000000,73.0,32,139.0,0.0,0.0,0.0,0.0,128
4,2000,Australian Open,Hard,101532.0,183.000000,71.0,32,39.0,0.0,0.0,0.0,0.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,2001,Wimbledon,Grass,103990.0,180.000000,75.0,19,59.0,0.0,0.0,0.0,0.0,64
1020,2001,Wimbledon,Grass,104014.0,184.331858,79.0,19,317.0,0.0,0.0,0.0,0.0,128
1021,2001,Wimbledon,Grass,104022.0,183.000000,73.0,19,85.0,0.0,0.0,0.0,0.0,16
1022,2001,Wimbledon,Grass,104053.0,188.000000,88.0,19,33.0,0.0,0.0,0.0,0.0,32


In [ ]:
data_features.loc[:,"age"] = final_data.loc[:,"year"] - final_data.loc[:,"birthdate"].dt.year

In [ ]:
import numpy as np

In [ ]:
players[players.id==float(i)]['ht'].values

array([183.])

In [ ]:
players

,id,name,birthdate,weight_kg,ht,ioc
0,101736.0,Andre Agassi,1970-04-29,80.0,180.000000,USA
1,103096.0,Arnaud Clement,1977-12-17,74.0,173.000000,FRA
2,102271.0,Hicham Arazi,1973-10-19,65.0,175.000000,MAR
3,101965.0,Wayne Ferreira,1971-09-15,85.0,185.000000,RSA
4,103324.0,Arnaud Di Pasquale,1979-02-11,80.0,185.000000,FRA
...,...,...,...,...,...,...
702,106109.0,Alex Bolt,1993-01-05,84.0,183.000000,AUS
703,106421.0,Daniil Medvedev,1996-02-11,81.0,198.000000,RUS
704,124187.0,Reilly Opelka,1997-08-28,100.0,211.000000,USA
705,106331.0,Christopher Oconnell,1994-06-03,NaN,184.331858,AUS


In [ ]:
data_features

,year,tourney_name,surface,tourney_id,id,rank,minutes_played,pj,pg,pp,round,ht,wg
0,2000,Australian Open,Hard,2000-580,101086.0,94.0,0.0,0.0,0.0,0.0,128,180.000000,82.0
1,2000,Australian Open,Hard,2000-580,101150.0,82.0,0.0,0.0,0.0,0.0,64,180.000000,80.0
2,2000,Australian Open,Hard,2000-580,101185.0,126.0,0.0,0.0,0.0,0.0,64,188.000000,88.0
3,2000,Australian Open,Hard,2000-580,101463.0,139.0,0.0,0.0,0.0,0.0,128,185.000000,73.0
4,2000,Australian Open,Hard,2000-580,101532.0,39.0,0.0,0.0,0.0,0.0,32,183.000000,71.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,2017,Australian Open,Hard,2017-580,124187.0,208.0,0.0,0.0,0.0,0.0,128,211.000000,100.0
8828,2017,Australian Open,Hard,2017-580,126094.0,152.0,0.0,0.0,0.0,0.0,64,188.000000,68.0
8829,2017,Australian Open,Hard,2017-580,126203.0,93.0,147.0,1.0,0.0,1.0,128,184.331858,NaN
8830,2017,Australian Open,Hard,2017-580,126207.0,108.0,0.0,0.0,0.0,0.0,64,184.331858,NaN


In [ ]:
data_features.loc[data_features.id==i,'ht']

351     183.0
607     183.0
730     183.0
856     183.0
988     183.0
1110    183.0
1231    183.0
1356    183.0
1489    183.0
1606    183.0
1737    183.0
1853    183.0
1986    183.0
2108    183.0
2232    183.0
2355    183.0
2490    183.0
2602    183.0
2728    183.0
Name: ht, dtype: float64

In [ ]:
players[players.id==103176]['ht'].values[0]

183.0

In [ ]:
for i in set(data_features.id):
    data_features.loc[data_features.id==i,'ht']=players[players.id==float(i)]['ht'].values[0]
    data_features.loc[data_features.id==i,'wg']=players[players.id==float(i)]['weight_kg'].values[0]

In [ ]:
players

,id,name,birthdate,weight_kg,ht,ioc
0,101736.0,Andre Agassi,1970-04-29,80.0,180.000000,USA
1,103096.0,Arnaud Clement,1977-12-17,74.0,173.000000,FRA
2,102271.0,Hicham Arazi,1973-10-19,65.0,175.000000,MAR
3,101965.0,Wayne Ferreira,1971-09-15,85.0,185.000000,RSA
4,103324.0,Arnaud Di Pasquale,1979-02-11,80.0,185.000000,FRA
...,...,...,...,...,...,...
702,106109.0,Alex Bolt,1993-01-05,84.0,183.000000,AUS
703,106421.0,Daniil Medvedev,1996-02-11,81.0,198.000000,RUS
704,124187.0,Reilly Opelka,1997-08-28,100.0,211.000000,USA
705,106331.0,Christopher Oconnell,1994-06-03,NaN,184.331858,AUS


In [ ]:
data_features

,year,tourney_name,surface,tourney_id,id,rank,minutes_played,pj,pg,pp,round
0,2000,Australian Open,Hard,2000-580,101086.0,94.0,0.0,0.0,0.0,0.0,128
1,2000,Australian Open,Hard,2000-580,101150.0,82.0,0.0,0.0,0.0,0.0,64
2,2000,Australian Open,Hard,2000-580,101185.0,126.0,0.0,0.0,0.0,0.0,64
3,2000,Australian Open,Hard,2000-580,101463.0,139.0,0.0,0.0,0.0,0.0,128
4,2000,Australian Open,Hard,2000-580,101532.0,39.0,0.0,0.0,0.0,0.0,32
...,...,...,...,...,...,...,...,...,...,...,...
8827,2017,Australian Open,Hard,2017-580,124187.0,208.0,0.0,0.0,0.0,0.0,128
8828,2017,Australian Open,Hard,2017-580,126094.0,152.0,0.0,0.0,0.0,0.0,64
8829,2017,Australian Open,Hard,2017-580,126203.0,93.0,147.0,1.0,0.0,1.0,128
8830,2017,Australian Open,Hard,2017-580,126207.0,108.0,0.0,0.0,0.0,0.0,64


In [ ]:
data_features

,year,tourney_name,tourney_level,surface,tourney_id,player,id,ht,age,rank,minutes_played,pj,pg,pp,round
0,2000,Australian Open,G,Hard,2000-580,Adrian Voinea,102437.0,185.0,25.448323,95.0,0.0,0.0,0.0,0.0,32
1,2000,Australian Open,G,Hard,2000-580,Albert Costa,102610.0,180.0,24.563997,20.0,0.0,0.0,0.0,0.0,128
2,2000,Australian Open,G,Hard,2000-580,Albert Portas,102287.0,188.0,26.171116,91.0,0.0,0.0,0.0,0.0,64
3,2000,Australian Open,G,Hard,2000-580,Alberto Berasategui,102238.0,173.0,26.554415,59.0,0.0,0.0,0.0,0.0,128
4,2000,Australian Open,G,Hard,2000-580,Alberto Martin,103252.0,175.0,21.409993,52.0,0.0,0.0,0.0,0.0,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8827,2017,Australian Open,G,Hard,2017-580,Tommy Haas,103163.0,188.0,38.789870,NaN,4593.0,32.0,21.0,11.0,128
8828,2017,Australian Open,G,Hard,2017-580,Victor Estrella,103607.0,173.0,36.457221,103.0,231.0,2.0,0.0,2.0,64
8829,2017,Australian Open,G,Hard,2017-580,Viktor Troicki,104678.0,193.0,30.932238,35.0,2740.0,17.0,9.0,8.0,32
8830,2017,Australian Open,G,Hard,2017-580,Yen Hsun Lu,104229.0,180.0,33.426420,60.0,2654.0,18.0,7.0,11.0,128


# Check integrality

In [ ]:
year = 2017
player = 'Roger Federer'
tourney = 'Australian Open'

In [ ]:
data_features[(data_features.player == player) & (data_features.tourney_name == tourney) & (data_features.year == year)]

,year,tourney_name,tourney_level,surface,tourney_id,player,id,ht,age,rank,minutes_played,pj,pg,pp,round
8812,2017,Australian Open,G,Hard,2017-580,Roger Federer,103819.0,185.0,35.441478,17.0,12211.0,94.0,81.0,13.0,1


In [ ]:
# Current
final_df[(final_df.tourney_name ==tourney) & (final_df.index.year == year) & (final_df.player == player)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,125.0,128,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,123.0,64,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,90.0,32,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,203.0,16,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,92.0,4,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,184.0,2,1
2017-01-16,2017-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,35.441478,17.0,217.0,1,1


In [ ]:
# History
final_df[(final_df.tourney_name ==tourney) & (final_df.index.year < year) & (final_df.player == player)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2000-01-17,2000-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,18.442163,62.0,125.0,128,1
2000-01-17,2000-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,18.442163,62.0,106.0,64,1
2001-01-15,2001-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,19.438741,29.0,132.0,128,1
2001-01-15,2001-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,19.438741,29.0,105.0,64,1
2002-01-14,2002-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,20.435318,12.0,126.0,128,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2012-01-16,2012-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,30.439425,3.0,222.0,2,0
2013-01-14,2013-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,31.436003,2.0,240.0,2,0
2014-01-13,2014-580,Australian Open,Hard,G,103819.0,Roger Federer,185.0,32.432580,6.0,144.0,2,0


In [ ]:
final_df[(final_df['round'] == 1) & (final_df.won == 1)]

,tourney_id,tourney_name,surface,tourney_level,id,player,ht,age,rank,minutes,round,won
tourney_date,,,,,,,,,,,,
2000-08-28,2000-560,US Open,Hard,G,103498.0,Marat Safin,193.0,20.585900,7.0,98.0,1,1
2000-01-17,2000-580,Australian Open,Hard,G,101736.0,Andre Agassi,180.0,29.719370,1.0,139.0,1,1
2000-06-26,2000-540,Wimbledon,Grass,G,101948.0,Pete Sampras,185.0,28.873374,3.0,178.0,1,1
2000-05-29,2000-520,Roland Garros,Clay,G,102856.0,Gustavo Kuerten,190.0,23.715264,5.0,224.0,1,1
2001-01-15,2001-580,Australian Open,Hard,G,101736.0,Andre Agassi,180.0,30.715948,6.0,106.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2016-01-18,2016-580,Australian Open,Hard,G,104925.0,Novak Djokovic,188.0,28.659822,1.0,173.0,1,1
2016-05-23,2016-520,Roland Garros,Clay,G,104925.0,Novak Djokovic,188.0,29.004791,1.0,183.0,1,1
2016-06-27,2016-540,Wimbledon,Grass,G,104918.0,Andy Murray,190.0,29.119781,2.0,168.0,1,1
